**Load Doccument**

In [ ]:
pip install langchain PyPDF2

In [2]:
import os
folder_path=r"C:\Users\user\Desktop\Medical ChatBot\data\Medical_book.pdf"
print(os.path.exists(folder_path))  # Should return True

True


In [3]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(path: str) -> List[Document]:
    documents = []
    if os.path.isfile(path):
        # Handle single file
        if path.endswith('.pdf'):
            loader = PyPDFLoader(path)
        elif path.endswith('.docx'):
            loader = Docx2txtLoader(path)
        else:
            print(f"Unsupported file type: {path}")
            return documents
        documents.extend(loader.load())
    elif os.path.isdir(path):
        # Handle directory
        for filename in os.listdir(path):
            file_path = os.path.join(path, filename)
            if filename.endswith('.pdf'):
                loader = PyPDFLoader(file_path)
            elif filename.endswith('.docx'):
                loader = Docx2txtLoader(file_path)
            else:
                print(f"Unsupported file type: {filename}")
                continue
            documents.extend(loader.load())
    else:
        print(f"Invalid path: {path}")
    return documents

documents = load_documents("data/Medical_book.pdf")
print(f"Loaded {len(documents)} documents.")


Loaded 637 documents.


In [4]:

from langchain_community.document_loaders import PyMuPDFLoader
from io import BytesIO
from multiprocessing import Pool
import tempfile
from tqdm import tqdm

LOADER_MAPPING = {
    ".pdf": (PyMuPDFLoader, {})
}

def load_single_document(file_content: BytesIO, file_type: str) -> List[Document]:
    ext = "." + file_type.rsplit("/", 1)[1]

    with tempfile.NamedTemporaryFile(suffix=ext, delete=False) as temp_file:
        temp_file.write(file_content.getvalue())
        temp_file_path = temp_file.name

    if ext in LOADER_MAPPING:
        loader_class, loader_args = LOADER_MAPPING[ext]
        loader = loader_class(temp_file_path, **loader_args)
        results = loader.load()
        os.remove(temp_file_path)
        return results

    raise ValueError(f"Unsupported file extension '{ext}'")

def load_uploaded_documents(uploaded_files, uploaded_files_type, ignored_files: List[str] = []) -> List[Document]:
    with Pool(processes=os.cpu_count()) as pool:
        results = []
        with tqdm(total=len(uploaded_files), desc='Loading new documents', ncols=80) as progress_bar:
            for i, uploaded_file in enumerate(uploaded_files):
                file_type = uploaded_files_type[i]
                file_content = BytesIO(uploaded_file.read())
                docs = load_single_document(file_content, file_type)
                results.extend(docs)
                progress_bar.update()
    return results

def get_pdf_text(uploaded_files):
    ignored_files = []  # Add files to ignore if needed

    uploaded_files_list = [file for file in uploaded_files]
    uploaded_files_type = [file.mimetype for file in uploaded_files]
    results = load_uploaded_documents(uploaded_files_list, uploaded_files_type, ignored_files)
    return results


In [5]:
def get_text_chunks(results, chunk_size, chunk_overlap):
    texts = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    texts = text_splitter.split_documents(results)
    return texts


In [6]:
import os
import glob
from flask import Flask, render_template, request, redirect, url_for, flash, session
from langchain_community.document_loaders import PyMuPDFLoader
from typing import List
from io import BytesIO
from dotenv import load_dotenv
from multiprocessing import Pool
import tempfile
from tqdm import tqdm
from langchain.chains import ConversationalRetrievalChain
from langchain.docstore.document import Document
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_cohere import CohereEmbeddings
from langchain_groq import ChatGroq
import logging
import uuid
from langchain.schema import HumanMessage, AIMessage

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\user\AppData\Local\sagemaker\sagemaker\config.yaml


In [7]:

COHERE_API_KEY='30lAzH7VO1y9M00rSD5ns4gIIxicqxiMnCn1IycB'
EMBEDDINGS_MODEL_NAME='embed-english-v2.0'
persist_directory='Medical ChatBot\db'
os.environ['COHERE_API_KEY']=COHERE_API_KEY
os.environ['EMBEDDINGS_MODEL_NAME']=EMBEDDINGS_MODEL_NAME

from langchain.docstore.document import Document

def get_vectorstore(results, embeddings_model_name, persist_directory, client_settings, chunk_size, chunk_overlap):
    embeddings = CohereEmbeddings(
        model=os.getenv('EMBEDDINGS_MODEL_NAME'),
        cohere_api_key=os.getenv('COHERE_API_KEY')
    )

    if does_vectorstore_exist(persist_directory):
        db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=client_settings)
        if texts := get_text_chunks(results, chunk_size=chunk_size, chunk_overlap=chunk_overlap):
            db.add_documents(texts)  # Use `texts` directly as they are already `Document` objects
    else:
        texts = get_text_chunks(results, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        db = Chroma.from_documents(
            documents=texts,  # Use `texts` directly
            embedding=embeddings,
            persist_directory=persist_directory,
            client_settings=client_settings
        )
        db.persist()

    return db

# def get_vectorstore(results, embeddings_model_name, persist_directory, client_settings, chunk_size, chunk_overlap):
#     embeddings = CohereEmbeddings(
#         model=os.getenv('EMBEDDINGS_MODEL_NAME'),
#         cohere_api_key=os.getenv('COHERE_API_KEY')
#     )

#     if does_vectorstore_exist(persist_directory):
        
#         db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=CHROMA_SETTINGS)
#         collection = db.get()
#         texts = get_text_chunks(results, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#         if len(texts) > 0:
#             db.add_documents(texts)
#     else:
#         # Create and store locally vectorstore
        
#         texts = get_text_chunks(results, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#         documents = [Document(page_content=text, metadata={}) for text in texts]
#         db = Chroma.from_documents(
#             documents=documents,
#             embedding=embeddings,
#             persist_directory=persist_directory,
#             client_settings=CHROMA_SETTINGS
#         )
#         #db = Chroma.from_documents(texts,embeddings, persist_directory=persist_directory, client_settings=CHROMA_SETTINGS)
#         db.add_documents(texts)

#     return db

In [8]:
def does_vectorstore_exist(persist_directory: str) -> bool:
    """
    Checks if vectorstore exists
    """
    if os.path.exists(os.path.join(persist_directory, 'index')):
        if os.path.exists(os.path.join(persist_directory, 'chroma-collections.parquet')) and os.path.exists(
                os.path.join(persist_directory, 'chroma-embeddings.parquet')):
            list_index_files = glob.glob(os.path.join(persist_directory, 'index/*.bin'))
            list_index_files += glob.glob(os.path.join(persist_directory, 'index/*.pkl'))
            # At least 1 documents are needed in a working vectorstore
            if len(list_index_files) > 0:
                logger.info("Vectorstore exists")
                return True
    return False

In [9]:
def get_text_chunks(results, chunk_size, chunk_overlap):
    texts = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    texts = text_splitter.split_documents(results)
    return texts


In [10]:

GROQ_API_KEY='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'
os.environ["GROQ_API_KEY"]='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'
llm= ChatGroq(
            model='llama3-8b-8192',
            temperature=0.6,
            api_key=os.getenv("GROQ_API_KEY")
        )

In [11]:
import mimetypes
from io import BytesIO

# Path to the file
file_path = r"C:\Users\user\Desktop\Medical ChatBot\data\Medical_book.pdf"

# Create a file-like object with mimetype
with open(file_path, 'rb') as f:
    file_content = BytesIO(f.read())
    file_content.name = file_path.split('\\')[-1]  # Set a file name attribute
    file_content.mimetype = mimetypes.guess_type(file_path)[0]  # Infer mimetype

# Call the function with the file object
uploaded_files = [file_content]
results = get_pdf_text(uploaded_files)




Loading new documents:   0%|                              | 0/1 [00:00<?, ?it/s]c:\Users\user\anaconda3\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 0 of document C:\Users\user\AppData\Local\Temp\tmpzpl7bq1t.pdf
  warnings.warn(
Loading new documents: 100%|██████████████████████| 1/1 [00:04<00:00,  4.02s/it]


In [12]:
text_chunks = get_text_chunks(results=results, chunk_size=500, chunk_overlap=20)

In [13]:
COHERE_API_KEY='30lAzH7VO1y9M00rSD5ns4gIIxicqxiMnCn1IycB'
EMBEDDINGS_MODEL_NAME='embed-english-v2.0'

In [14]:


persist_directory='db'
os.environ['COHERE_API_KEY']=COHERE_API_KEY
os.environ['EMBEDDINGS_MODEL_NAME']=EMBEDDINGS_MODEL_NAME

from langchain.docstore.document import Document

def get_vectorstore(results, embeddings_model_name, persist_directory, client_settings, chunk_size, chunk_overlap):
    embeddings = CohereEmbeddings(
        model=os.getenv('EMBEDDINGS_MODEL_NAME'),
        cohere_api_key=os.getenv('COHERE_API_KEY')
    )

    if does_vectorstore_exist(persist_directory):
        db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=client_settings)
        if texts := get_text_chunks(results, chunk_size=chunk_size, chunk_overlap=chunk_overlap):
            db.add_documents(texts)  # Use `texts` directly as they are already `Document` objects
    else:
        texts = get_text_chunks(results, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        db = Chroma.from_documents(
            documents=texts,  # Use `texts` directly
            embedding=embeddings,
            persist_directory=persist_directory,
            client_settings=client_settings
        )
        db.persist()

    return db

# def get_vectorstore(results, embeddings_model_name, persist_directory, client_settings, chunk_size, chunk_overlap):
#     embeddings = CohereEmbeddings(
#         model=os.getenv('EMBEDDINGS_MODEL_NAME'),
#         cohere_api_key=os.getenv('COHERE_API_KEY')
#     )

#     if does_vectorstore_exist(persist_directory):
        
#         db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=CHROMA_SETTINGS)
#         collection = db.get()
#         texts = get_text_chunks(results, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#         if len(texts) > 0:
#             db.add_documents(texts)
#     else:
#         # Create and store locally vectorstore
        
#         texts = get_text_chunks(results, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#         documents = [Document(page_content=text, metadata={}) for text in texts]
#         db = Chroma.from_documents(
#             documents=documents,
#             embedding=embeddings,
#             persist_directory=persist_directory,
#             client_settings=CHROMA_SETTINGS
#         )
#         #db = Chroma.from_documents(texts,embeddings, persist_directory=persist_directory, client_settings=CHROMA_SETTINGS)
#         db.add_documents(texts)

#     return db


In [19]:
embeddings = CohereEmbeddings(
        model=os.getenv('EMBEDDINGS_MODEL_NAME'),
        cohere_api_key=os.getenv('COHERE_API_KEY')
    )

In [32]:
pip install chromadb client

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement client (from versions: none)
ERROR: No matching distribution found for client


In [39]:
import chromadb
chroma_client = chromadb.Client()

# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = chroma_client.get_or_create_collection(name="my_collection")

# switch `add` to `upsert` to avoid adding the same documents every time
collection.upsert(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)

results = collection.query(
    query_texts=["This is a query document about florida"], # Chroma will embed this for you
    n_results=2 # how many results to return
)

print(results)


ValueError: An instance of Chroma already exists for ephemeral with different settings

In [29]:

vectordb = Chroma.from_documents(documents=text_chunks,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

ModuleNotFoundError: No module named 'pypika.dialects'

In [ ]:
!pip -q install chromadb  langchain tiktoken

In [28]:
!pip show chromadb

Name: chromadb
Version: 0.5.18
Summary: Chroma.
Home-page: https://github.com/chroma-core/chroma
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: bcrypt, build, chroma-hnswlib, fastapi, grpcio, httpx, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, orjson, overrides, posthog, pydantic, pypika, PyYAML, rich, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: chroma-migrate, langchain-chroma


In [17]:
from chromadb.config import Settings
CHROMA_SETTINGS = Settings(
        persist_directory=persist_directory,
        anonymized_telemetry=False
)
vectorstore = get_vectorstore(results=text_chunks, 
                              embeddings_model_name=EMBEDDINGS_MODEL_NAME,
                              persist_directory=persist_directory, 
                              client_settings=CHROMA_SETTINGS,
                              chunk_size=500, 
                              chunk_overlap=50)
    

ModuleNotFoundError: No module named 'pypika.dialects'

In [15]:
pip install pypika[dialects]


Note: you may need to restart the kernel to use updated packages.


In [16]:
retriever =db.as_retriever(search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

NameError: name 'db' is not defined